<a href="https://colab.research.google.com/github/ming-256/personal/blob/master/EPQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTANT


---


### 1.   At the top left of the screen, click "**Open in playground mode**"
###2.   Click accept on the popup
### 3.   At the top left of  the screen, click "Runtime" then "Run all"




In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

 The above lines are adding different functionalities to my program - *TensorFlow* allows me to use the modules and functions provided by TensorFlow, whilst *matplotlib* allows me to 'plot' or graph things allowing me to show certain things, making it much easier to explain my program.

In [ ]:
(training_images, training_labels), (testing_images, testing_labels) = keras.datasets.mnist.load_data()
training_images.shape

This line downloads the 70,000 images I will be using to train and test the neural network.
The first bracket contains 60,000 images and their associated labels, while the second bracket contains 10,000 images that will be used for evaluating the dataset.

The images and labels are stored as NumPy arrays, which are just a way of storing data. 
The labels are essentially 1D lists of 60,000 and 10,000 labels respectively whilst the images are somewhat harder to describe. The **training_images** can be thought of as containing 60,000 28x28 matrices, which is essentially 60,000 images, with each 28x28 matrix representing the pixels of the 28x28 image.

In [ ]:
plt.imshow(training_images[0], cmap='gray') #Change the number in the square brackets

This shows the images that are part of the MNIST dataset of handwritten digits, and are what this program will aim to identify. It consists of the digits 0-9, giving it 10 different digits total.
By changing the number in the square brackets above *plt.imshow(x_train **[0]** <------   )* and running the code, you will be able to see any of the 60,000 images in the training set.


As you can see from the axes, all the images are 28x28 pixels wide and high. 
This means the number of inputs to my neural network will be 28*28 = 784 inputs, since each pixel is a seperate input.

Currently, each pixel has a value between 0 and 255, which represents its greyscale value: how white the pixel is. 0 represents a completely white pixel, whilst 255 represents a completely black pixel. Anything between is a different shade of grey.

To train a convolutional neural network (CNN from now on) it is much easier to create a CNN if the pixels are in the range 0 to 1, as that way not only can they can be input into a wide range of 'activation functions' they will also be much more efficient than if the data wasn't 'normalized' (if the data is kept in the range 0-255). For example, the Sigmoid function will be unusable if the data isnt normalized.

Therefore, I will be dividing my data by 255 to get all the pixels in the interval (0,1), or, 'normalizing' my data.



---


Whilst this isn't the 'proper' way of normalizing data, it is sufficient in this case, as I am only classifying handwritten digits, and not trying to identify objects for self-driving cars or something important. Since in comparison, this is easy, the effects of using proper normalization techniques aren't as significant so it is fine to just divide by 255.

In [ ]:
training_images = training_images / 255.0
testing_images = testing_images / 255.0

The reason I divide by 255.0 and not just 255 is to do with Python: if I divide by 255, it will store the value as an integer, meaning the values will either be 0 or 1, causing a massive loss of information. It is the same as turning every pixel either completely black or pure white - which will be devestating, given that our images are only 28x28 pixels. Our CNN will find it much harder to identify the images, as it has much lass data to work with.

By dividing by 255.0, Python stores it as a float. i.e. a decimal, allowing it to be essentially anything between 0 and 1, like 0.0000001 and 0.938123841 etc.

In [ ]:
training_images = training_images.reshape(60000,28,28,1)
testing_images = testing_images.reshape(10000,28,28,1)

The two variables training_images and testing_images currently have no defined width, making them 2D.

These 2 lines specify to TensorFlow and Keras that the images are images with width 1, making them 3D images that are essentially 2D, as they have width 1. This is purely so the CNN will accept them as inputs, as Conv2D will only take 3D inputs.

In [ ]:
from keras import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Flatten, Dropout

I forgot to import the model type and layers I need to create the CNN, so I am doing that now. 

Sequential is a layer-by-layer model, where each layer influences the ones below it.

Conv2D, Dense, Maxpool2D, Flatten and Dropout are all types of layers, that comprise the CNN.

*   The Conv2D layer highlights important features in the image, and it does this through filters, . It can identify edges, sharpen images, reduce noise and more.

*   Maxpool2D reduces the size of the input by taking the max value in the size specified (2x2) and basically turning it into 1 pixel, without losing important features or patterns.

*   Flatten is a very important layer in CNNs, it converts the input into a 1D array that we can input into the Dense layers. It becomes a very long vector, which can be imagined as a long line of neurons from which connections can be easily made to other neurons in successive layers.

*   Dense layers are fully connected layers, where each neuron/node in the layer is connected with every single neuron in the previous layer. This is where the 'learning' happens: each connection's weights and biases can be adjusted as the CNN learns, in accordance with the optimizer and loss functions.

*   Dropout layers specify a fraction of neurons that will be 'dropped'. During training, random neurons will be ignored. This helps prevent overfitting as it stops surrounding neurons from relying too much on a specific neuron's output.
The network will become less reliant on specific neurons and their weights, hence allowing it to be able to generalize better and thus will perform better on new, testing data as it won't be overfitted.


In [ ]:
shape = (28,28,1)

These are the dimensions of the images - 28x28 pixels, and 1 wide so they are classified as 3D

This line is purely so it can be entered as a variable in the model below.

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape = shape))
model.add(MaxPool2D(2,2))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

This string of code is the framework of the CNN.
The order that the layers are added is the order that they will be executed, that is, top to bottom.

*   The first line creates a Sequential model - a linear stack of layers

*   The Conv2D layers can be thought of as taking a 3x3 square, and sliding it across the image, performing some calculations on the values of the pixels, and producing a new image with those values, called a feature map. The input_shape in the first layer tells the CNN what size data it should expect, and hence only needs to be put in once in the first layer. The filter size (32 and 64) represent the number of times this is done on the inputs. e.g. the first Conv2D layer will create 32 feature maps of size 26x26, since the edges will be cut off as they don't have a 3x3 around them.

*   The activation function 'relu' stands for the Rectified Linear Unit: It is a function that will output its input if it's > 0, and will output 0 if it's < 0. It is the most common activation function in CNNs. It is a very easy computation, compared to other activation functions and so reduces the time needed to train the model. It also has many other functionalities, but those are irrelevant for this CNN, as it is quite simple.

*   MaxPool2D then downsizes the dimensionality of the input, reducing the number of paramters so it takes less computational power, as well as increasing the CNN's abstractness, helping to prevent overfitting. It's input, (2,2), means it takes the maximum value of pixels in a 2x2 area.

*   The flatten layer then produces a 1D tensor/vector from the 2D data which can be input into the Dense layers, as they have no need for spacial information, unlike the previous few layers.

*   The first dense layer has 128 neurons with the relu activation function, and is connected to every neuron in the flatten layer

*   The dropout layer is configured to cause 25% of neurons to be ignored during training

*   The second dense layer has only 10 neurons with each connected to all 128 neurons from the previous layer - with each representing one of the digits from 0-9. This layer is the output: the one that shows us which digit the CNN thinks the image is. The output uses the softmax activation function because




In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(training_images,training_labels, epochs=10)

Epoch 1/10
   9/1875 [..............................] - ETA: 47s - loss: 2.2088 - accuracy: 0.2743

The compiler function initializes how the model is going to be put together, or, compiled. 

The loss function is a function that gives a mathematical output that tells the CNN how 'wrong' its prediction was. 

Simply put, the optimizer uses the loss function's outputs in order to tweak the connections between neurons in the Dense layers, so that the loss function will decrease. It does this through complex manipulations of gradients, learning rates, stochastic gradient descent and many more complicated methods, but can vaguely be thought of as a hiker trying to descend a mountain range: the further she descends, the more accurate the CNN.


The CNN will use the loss function and the optimizer in tandem to alter the weights and biases of the connections in order to improve the accuracy of the CNN. Alternatively, it can pictured as telling the CNN how much to change the strength of certain connections in the Dense layers (the fully conntected layers). 

Essentially, the CNN seeks to minimize the loss function (as you can see above, the loss value consistently decreases after every epoch). The optimizer provides the CNN with the means to do it.

The metric is what the CNN uses to evaluate its performance. Our CNN will give more weight to accuracy and less to other characteristics.

In [ ]:
a = model.evaluate(testing_images, testing_labels)
print("The accuracy of this CNN is ",a[1]*100,"%")

This evaluates the accuracy of the CNN, by giving it the 10,000 images of the testing set and asking the CNN what it thinks they are.

This is the final accuracy of the CNN on 10,000 images it has never seen before.



---



This allows you to ask the CNN to predict any image that you give it, by altering the number in the code above between 0 and 59999.

In [ ]:
number = 60 #<--- Change this number
plt.imshow(training_images[number].reshape(28,28), cmap='gray')
pred = model.predict(training_images[number].reshape(1,28,28,1))
print("The CNN's prediction is ",pred.argmax())